https://python.langchain.com/docs/tutorials/rag/

Build a Retrieval Augmented Generation (RAG) App: Part 1

In [ ]:
!pip install langchain langchain-chroma langchain-openai
!pip install beautifulsoup4
!pip install langchain-community langchain-text-splitters langgraph
!pip install faiss-cpu

In [ ]:
import getpass
import os

# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["OPENAI_API_KEY"] = getpass.getpass()

LANGCHAIN_TRACING_V2 = os.getenv("LANGCHAIN_TRACING_V2")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
print(f"LANGCHAIN_TRACING_V2: {LANGCHAIN_TRACING_V2}")
print(f"OPENAI_API_KEY: {os.environ.get('OPENAI_API_KEY', 'Not Set')}")

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(api_key=os.environ["OPENAI_API_KEY"])

In [ ]:
answer = llm.invoke("how can langsmith help with testing?")

In [ ]:
print(f"answer: {answer}")
print(f"Type: {type(answer)}")

In [ ]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

# Load and chunk contents of the blog
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

# Index chunks
_ = vector_store.add_documents(documents=all_splits)

# Define prompt for question-answering
# N.B. for non-US LangSmith endpoints, you may need to specify
# api_url="https://api.smith.langchain.com" in hub.pull.
prompt = hub.pull("rlm/rag-prompt")


# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

# Let's Do Some Chaining
- This is the key point! LangChain's power comes from chaining different components together. In the simple example above, you're just making a direct call to the LLM. There's no:
    - Prompt templates
    - Output parsers
    - Retrieval components
    - Multiple processing steps
    - Data transformations


## Templating 

What it's doing:  

1. Import the ChatPromptTemplate: This is LangChain's tool for creating structured conversation templates.

2. Create a conversation template: The ChatPromptTemplate.from_messages() creates a template that represents a multi-turn conversation with different roles:

```
"system": Sets up the AI's identity and behavior (with a placeholder {name})  
"human": First human message  
"ai": AI's response to establish context  
"human": Second human message (with a placeholder {user_input})  
```

3. Use placeholders: The curly braces {name} and {user_input} are variables that will be filled in later.


4. Invoke the template: When you call template.invoke() with the dictionary of values:

- {name} gets replaced with "Bob"
- {user_input} gets replaced with "What is your name?"

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI bot. Your name is {name}."),
    ("human", "Hello, how are you doing?"),
    ("ai", "I'm doing well, thanks!"),
    ("human", "{user_input}"),
])

prompt_value = template.invoke(
    {
        "name": "Bob",
        "user_input": "What is your name?"
    }
)


In [ ]:
for msg in prompt_value.messages:
  print(type(msg).__name__, ":", msg.content)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class technical documentation writer."), # system instructions
    ("user", "{input}")
])

In [ ]:
for msg in prompt.messages:
  print(type(msg).__name__, ":", msg)

![chaining.png](../Assets/images/chaining.png)

## Chaining

In [ ]:
print(prompt)

Think of it like a pipeline or assembly line:

The | symbol = "then" or "pipe to"
prompt = Your formatted question/template
llm = The AI model that generates answers
So chain = prompt | llm means:

Take the prompt → THEN → send it to the LLM


In [ ]:
chain = prompt | llm
## pass the prompt to the LLM

In [ ]:
#chain.first shows you the first component in your chain.

print(chain.middle)

In [ ]:
chain_result = chain.invoke({"input": "how can langsmith help with testing?"})

"system": "You are a world class technical documentation writer."

"user", "how can langsmith help with testing?"

In [ ]:
print(chain_result.content) #This line displays the actual text response from the AI.

### This displays the technical information about the AI's response - all the "behind the scenes" details.

In [ ]:
print(chain_result.response_metadata)

In [ ]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

### This creates a 3-step chain instead of the previous 2-step chain. Now you have:


### The 3 Steps:
- prompt → Format the question
- llm → Get AI response (complex object)
- output_parser → Clean up the response (convert to simple string)

### Think of it like a car wash:

- Step 1: Prep the car (format prompt)
- Step 2: Wash the car (get AI response)
- Step 3: Dry and polish (clean up the text)

In [ ]:
chain = prompt | llm | output_parser

In [ ]:
chain_result = chain.invoke({"input": "how can langsmith help with testing?"})

In [ ]:
print(chain_result)

# Where is the retrieval from <u>**R**</u>AG

In [ ]:
#Imports a tool that can read websites
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/")

docs = loader.load() # The docs variable now contains all that website content, ready to be processed further!

In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings() # Creates an instance of the embeddings model that's ready to use

### Take Langchains website content and create a searchable database of it.

Think of it like creating a smart library:

- Text Splitter: Takes a huge book and breaks it into individual pages/chapters
- Embeddings: Creates a "topic card" for each page that describes what it's about
- FAISS: Organizes all those topic cards so you can quickly find relevant pages

In [ ]:
#This code takes your website content and creates a searchable database of it.

from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [ ]:
print(documents)

### This creates a specialized chain that can answer questions using specific documents as context

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
# create_stuff_documents_chain :
#  Create a chain for passing a list of Documents to a model.

prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""", output_parser = output_parser)

document_chain = create_stuff_documents_chain(llm, prompt)
# document_chain = prompt | llm /

### Think of it like a research assistant:

- Before: AI answers from its general knowledge
- Now: AI gets a stack of specific research papers and must answer ONLY using those papers

In [ ]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

### This line runs the complete RAG system you've been building throughout the notebook!

In [ ]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})

In [ ]:
print(response["answer"])

# LangSmith offers several features that can help with testing:...

In [ ]:
response = retrieval_chain.invoke({"input": "how can use it?"})
print(response["answer"])